# Tutorial

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/massquantity/LibRecommender/blob/master/examples/tutorial.ipynb)
[![View in doc](https://img.shields.io/badge/document-tutorial-ffdfba)](https://librecommender.readthedocs.io/en/latest/tutorial.html)

This tutorial will walk you through the comprehensive process of training a model in LibRecommender, i.e. **data processing -> feature engineering -> training -> evaluate -> save/load -> retrain**. We will use [Wide & Deep](https://arxiv.org/pdf/1606.07792.pdf) as the example algorithm. 

First make sure the latest LibRecommender has been installed:

In [ ]:
!pip install -U LibRecommender

For how to deploy a trained model in LibRecommender, see [Serving Guide](https://librecommender.readthedocs.io/en/latest/serving_guide/python.html).

**NOTE**: If you encounter errors like `Variables already exist, disallowed...`, just call `tf.compat.v1.reset_default_graph()` first.

## Load data

In this tutorial we willl use the [MovieLens 1M](https://grouplens.org/datasets/movielens/1m/) dataset. The following code will load the data into `pandas.DataFrame` format. If the data does not exist locally, it will be downloaded at first.

In [5]:
import random
import warnings
import zipfile
from pathlib import Path

import pandas as pd
import tensorflow as tf
import tqdm
warnings.filterwarnings("ignore")

In [2]:
def load_ml_1m():
    # download and extract zip file
    tf.keras.utils.get_file(
        "ml-1m.zip", 
        "http://files.grouplens.org/datasets/movielens/ml-1m.zip", 
        cache_dir=".", 
        cache_subdir=".", 
        extract=True,
    )
    # read and merge data into same table
    cur_path = Path(".").absolute()
    ratings = pd.read_csv(
        cur_path / "ml-1m" / "ratings.dat", 
        sep="::", 
        usecols=[0, 1, 2, 3], 
        names=["user", "item", "rating", "time"],
    )
    users = pd.read_csv(
        cur_path / "ml-1m" / "users.dat", 
        sep="::",
        usecols=[0, 1, 2, 3], 
        names=["user", "sex", "age", "occupation"],
    )
    items = pd.read_csv(
        cur_path / "ml-1m" / "movies.dat", 
        sep="::",
        usecols=[0, 2], 
        names=["item", "genre"],
        encoding="iso-8859-1",
    )
    items[["genre1", "genre2", "genre3"]] = (
        items["genre"].str.split(r"|", expand=True).fillna("missing").iloc[:, :3]
    )
    items.drop("genre", axis=1, inplace=True)
    data = ratings.merge(users, on="user").merge(items, on="item")
    data.rename(columns={"rating": "label"}, inplace=True)
    # random shuffle data
    data = data.sample(frac=1, random_state=42).reset_index(drop=True)
    return data

In [3]:
data = load_ml_1m()
print("data shape:", data.shape)

data shape: (1000209, 10)


In [4]:
data.iloc[random.choices(range(len(data)), k=10)]  # randomly select 10 rows

,user,item,label,time,sex,age,occupation,genre1,genre2,genre3
897381,4127,3699,3,965356931,M,50,17,Adventure,Drama,Romance
615720,3119,1240,4,969393440,M,45,1,Action,Sci-Fi,Thriller
28966,2966,2581,4,971137483,F,25,4,Comedy,Romance,missing
691341,1938,2105,4,974695701,F,25,2,Action,Adventure,Fantasy
685815,1522,2968,4,974745539,M,35,20,Adventure,Fantasy,Sci-Fi
925314,2816,480,4,972682268,M,25,7,Action,Adventure,Sci-Fi
690597,5780,788,4,958154511,M,18,17,Comedy,Fantasy,Romance
383093,1110,593,4,974919464,F,56,6,Drama,Thriller,missing
686437,4829,1653,5,1007833084,M,25,0,Drama,Sci-Fi,Thriller
246925,4995,2985,3,962596185,M,50,20,Action,Crime,Sci-Fi


Now we have about 1 million data. In order to perform evaluation after training, we need to split the data into train, eval and test data first. In this tutorial we will simply use `random_split`. For other ways of splitting data, see [Data Processing](https://librecommender.readthedocs.io/en/latest/user_guide/data_processing.html).

**For now, We will only use first half data for training. Later we will use the rest data to retrain the model.**

## Process Data & Features

In [7]:
from libreco.data import random_split

# split data into three folds for training, evaluating and testing
first_half_data = data[: (len(data) // 2)]
train_data, eval_data, test_data = random_split(first_half_data, multi_ratios=[0.8, 0.1, 0.1], seed=42)

The data contains some categorical features such as "sex" and "genre", as well as a numerical feature "age". In LibRecommender we use `sparse_col` to represent categorical features and `dense_col` to represent numerical features. So one should specify the column information and then use `DatasetFeat.build_*` functions to process the data.

In [8]:
print("first half data shape:", first_half_data.shape)

first half data shape: (500104, 10)


In [9]:
from libreco.data import DatasetFeat

sparse_col = ["sex", "occupation", "genre1", "genre2", "genre3"]
dense_col = ["age"]
user_col = ["sex", "age", "occupation"]
item_col = ["genre1", "genre2", "genre3"]

train_data, data_info = DatasetFeat.build_trainset(train_data, user_col, item_col, sparse_col, dense_col)
eval_data = DatasetFeat.build_evalset(eval_data)
test_data = DatasetFeat.build_testset(test_data)

"user_col" means features belong to user, and "item_col" means features belong to item. Note that the column numbers should match, i.e. `len(sparse_col) + len(dense_col) == len(user_col) + len(item_col)`.

In [10]:
print(data_info)

n_users: 6040, n_items: 3576, data density: 1.8523 %


## Training the Model

Now with all the data and features prepared, we can start training the model! 

Since as its name suggests, the `Wide & Deep` algorithm has wide and deep parts, and they use different optimizers. So we should specify the learning rate separately by using a dict: `{"wide": 0.01, "deep": 3e-4}`. For other model hyperparameters, see API reference of [WideDeep](https://librecommender.readthedocs.io/en/latest/api/algorithms/wide_deep.html).

In this example we treat all the samples in data as positive samples, and perform negative sampling. This is called "implicit data".

In [11]:
from libreco.algorithms import WideDeep

In [12]:
model = WideDeep(
    task="ranking",
    data_info=data_info,
    embed_size=16,
    n_epochs=2,
    loss_type="cross_entropy",
    lr={"wide": 0.05, "deep": 7e-4},
    batch_size=2048,
    use_bn=True,
    hidden_units=(128, 64, 32),
)

model.fit(
    train_data,
    neg_sampling=True,  # perform negative sampling on training and eval data
    verbose=2,
    shuffle=True,
    eval_data=eval_data,
    metrics=["loss", "roc_auc", "precision", "recall", "ndcg"],
)

Training start time: 2023-04-06 15:12:45
Instructions for updating:
Colocations handled automatically by placer.


2023-04-06 15:12:45.758683: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-06 15:12:45,844 - WARNING - From /home/massquantity/miniconda3/lib/python3.8/site-packages/keras/layers/normalization/batch_normalization.py:562: _colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


total params: 192,413 | embedding params: 165,109 | network params: 27,304


2023-04-06 15:12:46.174116: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
train: 100%|██████████████████████████████████████████████████████| 391/391 [00:02<00:00, 134.68it/s]


Epoch 1 elapsed: 2.905s
	 train_loss: 0.959
random neg item sampling elapsed: 0.024s


...

	 eval log_loss: 0.5823
	 eval roc_auc: 0.8032
	 eval precision@10: 0.0236
	 eval recall@10: 0.0339
	 eval ndcg@10: 0.1001


train: 100%|██████████████████████████████████████████████████████| 391/391 [00:02<00:00, 156.01it/s]


Epoch 2 elapsed: 2.508s
	 train_loss: 0.499


eval_listwise: 100%|████████████████████████████████████████████| 2797/2797 [00:10<00:00, 256.00it/s]


	 eval log_loss: 0.4769
	 eval roc_auc: 0.8488
	 eval precision@10: 0.0332
	 eval recall@10: 0.0523
	 eval ndcg@10: 0.1376


We've trained the model for 2 epochs and evaluated the performance on the eval data during training. Next we can evaluate on the *independent* test data.

In [13]:
from libreco.evaluation import evaluate

evaluate(
    model=model, 
    data=test_data, 
    neg_sampling=True,  # perform negative sampling on test data
    metrics=["loss", "roc_auc", "precision", "recall", "ndcg"],
)

random neg item sampling elapsed: 0.025s


eval_listwise: 100%|████████████████████████████████████████████| 2834/2834 [00:10<00:00, 278.12it/s]


{'loss': 0.4782908669403157,
 'roc_auc': 0.8483713737644527,
 'precision': 0.031268748897123694,
 'recall': 0.04829594849021039,
 'ndcg': 0.12866793895121623}

## Make Recommendation

The recommend part is pretty straightforward. You can make recommendation for one user or a batch of users.

In [14]:
model.recommend_user(user=1, n_rec=3)

{1: array([ 364, 3751, 2858])}

In [16]:
model.recommend_user(user=[1, 2, 3], n_rec=3)

{1: array([ 364, 3751, 2858]),
 2: array([1617,  608,  912]),
 3: array([ 589, 2571, 1200])}

You can also make recommdation based on specific user features.

In [19]:
model.recommend_user(user=1, n_rec=3, user_feats={"sex": "M", "age": 33})

{1: array([2716,  589, 2571])}

In [20]:
model.recommend_user(user=1, n_rec=3, user_feats={"occupation": 17})

{1: array([2858, 1210, 1580])}

## Save, Load and Inference

When saving the model, we should also save the `data_info` for feature information.

In [21]:
data_info.save("model_path", model_name="wide_deep")
model.save("model_path", model_name="wide_deep")

Then we can load the model and make recommendation again.

In [22]:
tf.compat.v1.reset_default_graph()  # need to reset graph in TensorFlow1

In [23]:
from libreco.data import DataInfo

loaded_data_info = DataInfo.load("model_path", model_name="wide_deep")
loaded_model = WideDeep.load("model_path", model_name="wide_deep", data_info=loaded_data_info)
loaded_model.recommend_user(user=1, n_rec=3)

total params: 192,413 | embedding params: 165,109 | network params: 27,304


{1: array([ 364, 3751, 2858])}

## Retrain the Model with New Data

Remember that we split the original `MovieLens 1M` data into two parts in the first place? We will treat the **second half** of the data as our new data and retrain the saved model with it. In real-world recommender systems, data may be generated every day, so it is inefficient to train the model from scratch every time we get some new data.

In [24]:
second_half_data = data[(len(data) // 2) :]
train_data, eval_data = random_split(second_half_data, multi_ratios=[0.8, 0.2])

In [25]:
print("second half data shape:", second_half_data.shape)

second half data shape: (500105, 10)


The data processing is similar, except that we should use `merge_trainset()` and `merge_evalset()` in DatasetFeat.

The purpose of these functions is combining information from old data with that from new data, especially for the possible new users/items from new data. For more details, see [Model Retrain](https://librecommender.readthedocs.io/en/latest/user_guide/model_retrain.html).

In [26]:
# pass `loaded_data_info` and get `new_data_info`
train_data, new_data_info = DatasetFeat.merge_trainset(train_data, loaded_data_info, merge_behavior=True)
eval_data = DatasetFeat.merge_evalset(eval_data, new_data_info)  # use new_data_info

Then we construct a new model, and call `rebuild_model` method to assign the old variables into the new model.

In [27]:
tf.compat.v1.reset_default_graph()  # need to reset graph in TensorFlow1

In [28]:
new_model = WideDeep(
    task="ranking",
    data_info=new_data_info,  # pass new_data_info
    embed_size=16,
    n_epochs=2,
    loss_type="cross_entropy",
    lr={"wide": 0.01, "deep": 1e-4},
    batch_size=2048,
    use_bn=True,
    hidden_units=(128, 64, 32),
)

new_model.rebuild_model(path="model_path", model_name="wide_deep", full_assign=True)

total params: 194,164 | embedding params: 166,860 | network params: 27,304


Finally, the training and recommendation parts are the same as before.

In [29]:
new_model.fit(
    train_data, 
    neg_sampling=True,
    verbose=2, 
    shuffle=True, 
    eval_data=eval_data,
    metrics=["loss", "roc_auc", "precision", "recall", "ndcg"],
)

Training start time: 2023-04-06 15:18:29


train: 100%|██████████████████████████████████████████████████████| 391/391 [00:02<00:00, 136.50it/s]


Epoch 1 elapsed: 2.867s
	 train_loss: 0.4867
random neg item sampling elapsed: 0.058s


eval_listwise: 100%|████████████████████████████████████████████| 2981/2981 [00:11<00:00, 262.03it/s]


	 eval log_loss: 0.4482
	 eval roc_auc: 0.8708
	 eval precision@10: 0.0985
	 eval recall@10: 0.0710
	 eval ndcg@10: 0.3062


train: 100%|██████████████████████████████████████████████████████| 391/391 [00:02<00:00, 141.23it/s]


Epoch 2 elapsed: 2.770s
	 train_loss: 0.472


eval_listwise: 100%|████████████████████████████████████████████| 2981/2981 [00:10<00:00, 275.00it/s]


	 eval log_loss: 0.4416
	 eval roc_auc: 0.8741
	 eval precision@10: 0.1031
	 eval recall@10: 0.0738
	 eval ndcg@10: 0.3168


In [30]:
new_model.recommend_user(user=1, n_rec=3)

{1: array([ 364, 2858, 1210])}

In [31]:
new_model.recommend_user(user=[1, 2, 3], n_rec=3)

{1: array([ 364, 2858, 1210]),
 2: array([ 608, 1617, 1233]),
 3: array([ 589, 2571, 1387])}

**This completes our tutorial!**

+ For more examples, see the [examples](https://github.com/massquantity/LibRecommender/tree/master/examples) folder on GitHub. 

+ For more usages, please head to [User Guide](https://librecommender.readthedocs.io/en/latest/user_guide/index.html).

+ For serving a trained model, please head to [Python Serving Guide](https://librecommender.readthedocs.io/en/latest/serving_guide/python.html).